In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [57]:
def annotate_lst(df, var): # template_id, epitope, peptide, peptide_HLA, HLA
    if all(df.applymap(type)[var] == list):
        #dct = df.groupby('gem')[var].apply(lambda x: np.unique([z for y in x for z in y])).to_dict()
        dct = df.groupby('gem')[var].apply(list).to_dict()
    else:
        dct = df.groupby('gem')[var].unique().to_dict()
    return df.gem.map(dct)

In [60]:
def annotate_peptide_specificities():
    return pd.merge(mhc_df, specificity_df, how='left', on='barcode')

def annotate_sample_specificities():
    return pd.merge(cd8_df, cd8_specificity_df, how='left', on='barcode')

# Input

In [52]:
MAP = '/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/exp9_MHC_IONTORRENT/mapping/KMA-1t1/output/mapping.clean.gz'

In [48]:
specificity_annotations = '/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/exp9_MHC_IONTORRENT/barcode_library/barcode_specificity_annotations.xlsx'

In [2]:
INPUT = '/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/exp9_CAT_IONTORRENT_KMA_AKB/tables/tcr_barcode.cleaned.csv'

In [65]:
np.unique([[1,2],[2,3],[1,2]])

array([1, 2, 3])

In [86]:
x = [[1,2],[2,3],[1,2]]
np.unique([z for y in x for z in y])

array([1, 2, 3])

In [71]:
pd.Series([[1,2],[2,3],[1,2]],index=['a','b','a'])

a    [1, 2]
b    [2, 3]
a    [1, 2]
dtype: object

In [72]:
cd8_df.groupby('gem')['HLA'].apply(list)

gem
AAACCTGAGCACCGTC-1    [[A0201, A2501, B0702, B3501, C0401, C0702], [...
AAACGGGTCCCAGGTG-1    [[A0201, A0301, B1402, B4001, C0304, C0802], [...
AAAGATGAGCTCTCGG-1         [[A0301, A2402, B0702, B0801, C0701, C0702]]
AAAGATGTCCGAAGAG-1         [[A0101, A3201, B0702, B1401, C0702, C0802]]
AAAGTAGAGAAGCCCA-1    [[A0201, A0301, B1402, B4001, C0304, C0802], [...
                                            ...                        
TTTGCGCGTAATCGTC-1         [[A0301, A2402, B0702, B0801, C0701, C0702]]
TTTGGTTAGTGCCAGA-1         [[A0201, A0301, B1402, B4001, C0304, C0802]]
TTTGGTTCACGCGAAA-1         [[A0301, A2402, B0702, B0801, C0701, C0702]]
TTTGTCAAGGGATGGG-1    [[A0201, A2501, B0702, B3501, C0401, C0702], [...
TTTGTCATCTCCAACC-1         [[A0201, A0301, B1402, B4001, C0304, C0802]]
Name: HLA, Length: 557, dtype: object

# Load

In [53]:
map_df = pd.read_csv(MAP)

In [124]:
specificity_df = pd.read_excel(specificity_annotations, sheet_name=0, names=['barcode', 'peptide', 'HLA', 'epitope']) #skiprows=1
cd8_specificity_df = pd.read_excel(specificity_annotations, sheet_name=1, names=['barcode','sample_id','HLA_A','HLA_B','HLA_C','comment'])

specificity_df['peptide'] = specificity_df.peptide.str.split("_", expand=True)[0]
specificity_df['peptide_HLA'] = specificity_df.peptide + ' ' + specificity_df.HLA #.str.split('_').str[0]

cd8_specificity_df.fillna('', inplace=True)
cd8_specificity_df['HLA'] = cd8_specificity_df['HLA_A'] + ', ' + cd8_specificity_df['HLA_B'] + ', ' + cd8_specificity_df['HLA_C']
cd8_specificity_df['HLA'] = cd8_specificity_df.HLA.str.split(', ').apply(lambda x: [i for i in x if i!= ''])

CDX_NOMENCLATURE = cd8_specificity_df.barcode

In [125]:
cd8_specificity_df.HLA

0    [A0101, A3201, B0702, B1401, C0702, C0802]
1    [A0301, A2402, B0702, B0801, C0701, C0702]
2    [A0201, A2501, B0702, B3501, C0401, C0702]
3    [A0201, A0301, B1402, B4001, C0304, C0802]
4    [A0101, A3201, B0702, B1401, C0702, C0802]
5    [A0301, A2402, B0702, B0801, C0701, C0702]
6    [A0201, A2501, B0702, B3501, C0401, C0702]
7    [A0201, A0301, B1402, B4001, C0304, C0802]
Name: HLA, dtype: object

In [3]:
df = pd.read_csv(INPUT)

In [55]:
map_df

,uncertainty,score,t_alignment_start,t_alignment_end,template_id,read_header,query_id,gem,credible_alignment,alignment_length,barcode,sample
0,1,86,0,137,A1072B303_sample,O9A3T:00253:02876 BX:Z:TTTACTGCAAGCGCTC-1,O9A3T:00253:02876,TTTACTGCAAGCGCTC-1,True,137,A1072B303,sample
1,1,113,0,137,A1072B303_sample,O9A3T:00356:00520 BX:Z:TTGACTTGTCTCTTAT-1,O9A3T:00356:00520,TTGACTTGTCTCTTAT-1,True,137,A1072B303,sample
2,1,68,17,137,A1072B303_sample,O9A3T:00610:06658 BX:Z:TTCTCAAGTAAACCTC-1,O9A3T:00610:06658,TTCTCAAGTAAACCTC-1,True,120,A1072B303,sample
3,1,95,0,137,A1072B303_sample,O9A3T:00573:05912 BX:Z:TGTCCCAAGATCTGAA-1,O9A3T:00573:05912,TGTCCCAAGATCTGAA-1,True,137,A1072B303,sample
4,1,92,0,137,A1072B303_sample,O9A3T:01715:05174 BX:Z:TGTCCCAAGATCTGAA-1,O9A3T:01715:05174,TGTCCCAAGATCTGAA-1,True,137,A1072B303,sample
...,...,...,...,...,...,...,...,...,...,...,...,...
739466,1,112,0,137,A1079B318_sample,O9A3T:00214:06437 BX:Z:CGCGGTACAAAGGTGC-1,O9A3T:00214:06437,CGCGGTACAAAGGTGC-1,True,137,A1079B318,sample
739467,1,112,0,137,A1079B318_sample,O9A3T:01269:03677 BX:Z:CCATGTCCATCCAACA-1,O9A3T:01269:03677,CCATGTCCATCCAACA-1,True,137,A1079B318,sample
739468,1,100,0,137,A1079B318_sample,O9A3T:01199:03529 BX:Z:CATGCCTTCCCAAGAT-1,O9A3T:01199:03529,CATGCCTTCCCAAGAT-1,True,137,A1079B318,sample
739469,1,112,0,137,A1079B318_sample,O9A3T:01702:00540 BX:Z:GTTACAGTCGGATGGA-1,O9A3T:01702:00540,GTTACAGTCGGATGGA-1,True,137,A1079B318,sample


In [56]:
cd8_df = map_df[map_df.template_id.str.split('_', expand=True)[0].isin(CDX_NOMENCLATURE)].copy()
mhc_df = map_df[~map_df.template_id.str.split('_', expand=True)[0].isin(CDX_NOMENCLATURE)].copy()

In [61]:
cd8_df = annotate_sample_specificities()
mhc_df = annotate_peptide_specificities()

In [116]:
x = [['a','b'],['b','c'],['a','b']]
x.sort()

In [117]:
[k for k,_ in itertools.groupby(x)]

[['a', 'b'], ['b', 'c']]

In [107]:
cd8_df.groupby('gem')['HLA'].apply(list).apply(lambda x: [k for k, m in itertools.groupby(x)])

gem
AAACCTGAGCACCGTC-1    [[A0201, A2501, B0702, B3501, C0401, C0702]]
AAACGGGTCCCAGGTG-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
AAAGATGAGCTCTCGG-1    [[A0301, A2402, B0702, B0801, C0701, C0702]]
AAAGATGTCCGAAGAG-1    [[A0101, A3201, B0702, B1401, C0702, C0802]]
AAAGTAGAGAAGCCCA-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
                                          ...                     
TTTGCGCGTAATCGTC-1    [[A0301, A2402, B0702, B0801, C0701, C0702]]
TTTGGTTAGTGCCAGA-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
TTTGGTTCACGCGAAA-1    [[A0301, A2402, B0702, B0801, C0701, C0702]]
TTTGTCAAGGGATGGG-1    [[A0201, A2501, B0702, B3501, C0401, C0702]]
TTTGTCATCTCCAACC-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
Name: HLA, Length: 557, dtype: object

In [81]:
cd8_df.groupby('gem')['HLA'].apply(list).apply(lambda k: list(k for k,_ in itertools.groupby(k)))

gem
AAACCTGAGCACCGTC-1    [[A0201, A2501, B0702, B3501, C0401, C0702]]
AAACGGGTCCCAGGTG-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
AAAGATGAGCTCTCGG-1    [[A0301, A2402, B0702, B0801, C0701, C0702]]
AAAGATGTCCGAAGAG-1    [[A0101, A3201, B0702, B1401, C0702, C0802]]
AAAGTAGAGAAGCCCA-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
                                          ...                     
TTTGCGCGTAATCGTC-1    [[A0301, A2402, B0702, B0801, C0701, C0702]]
TTTGGTTAGTGCCAGA-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
TTTGGTTCACGCGAAA-1    [[A0301, A2402, B0702, B0801, C0701, C0702]]
TTTGTCAAGGGATGGG-1    [[A0201, A2501, B0702, B3501, C0401, C0702]]
TTTGTCATCTCCAACC-1    [[A0201, A0301, B1402, B4001, C0304, C0802]]
Name: HLA, Length: 557, dtype: object

In [119]:
cd8_df.gem.map(cd8_df.groupby('gem')['HLA'].apply(lambda x: np.unique([z for y in x for z in y])).to_dict())

0        [A0101, A3201, B0702, B1401, C0702, C0802]
1        [A0101, A3201, B0702, B1401, C0702, C0802]
2        [A0101, A3201, B0702, B1401, C0702, C0802]
3        [A0101, A3201, B0702, B1401, C0702, C0802]
4        [A0101, A3201, B0702, B1401, C0702, C0802]
                            ...                    
19532    [A0201, A0301, B1402, B4001, C0304, C0802]
19533    [A0201, A0301, B1402, B4001, C0304, C0802]
19534    [A0201, A0301, B1402, B4001, C0304, C0802]
19535    [A0201, A0301, B1402, B4001, C0304, C0802]
19536    [A0201, A0301, B1402, B4001, C0304, C0802]
Name: gem, Length: 19537, dtype: object

In [118]:
cd8_df.gem.map(cd8_df.groupby('gem')['HLA'].apply(lambda x: np.unique(x)).to_dict())

0        [[A0101, A3201, B0702, B1401, C0702, C0802]]
1        [[A0101, A3201, B0702, B1401, C0702, C0802]]
2        [[A0101, A3201, B0702, B1401, C0702, C0802]]
3        [[A0101, A3201, B0702, B1401, C0702, C0802]]
4        [[A0101, A3201, B0702, B1401, C0702, C0802]]
                             ...                     
19532    [[A0201, A0301, B1402, B4001, C0304, C0802]]
19533    [[A0201, A0301, B1402, B4001, C0304, C0802]]
19534    [[A0201, A0301, B1402, B4001, C0304, C0802]]
19535    [[A0201, A0301, B1402, B4001, C0304, C0802]]
19536    [[A0201, A0301, B1402, B4001, C0304, C0802]]
Name: gem, Length: 19537, dtype: object

In [63]:
annotate_lst(cd8_df, 'HLA')

0             [[A0101, A3201, B0702, B1401, C0702, C0802]]
1        [[A0101, A3201, B0702, B1401, C0702, C0802], [...
2        [[A0101, A3201, B0702, B1401, C0702, C0802], [...
3        [[A0101, A3201, B0702, B1401, C0702, C0802], [...
4        [[A0101, A3201, B0702, B1401, C0702, C0802], [...
                               ...                        
19532    [[A0201, A0301, B1402, B4001, C0304, C0802], [...
19533    [[A0201, A0301, B1402, B4001, C0304, C0802], [...
19534    [[A0201, A0301, B1402, B4001, C0304, C0802], [...
19535    [[A0201, A0301, B1402, B4001, C0304, C0802], [...
19536    [[A0201, A0301, B1402, B4001, C0304, C0802], [...
Name: gem, Length: 19537, dtype: object

In [78]:
cd8_df[cd8_df.gem.isin(['AAACCTGAGCACCGTC-1','AAACGGGTCCCAGGTG-1','AAAGATGAGCTCTCGG-1'])]

,uncertainty,score,t_alignment_start,t_alignment_end,template_id,read_header,query_id,gem,credible_alignment,alignment_length,barcode,sample,sample_id,HLA_A,HLA_B,HLA_C,comment,HLA
5401,1,110,0,137,A1072B310_sample,O9A3T:00502:02153 BX:Z:AAAGATGAGCTCTCGG-1,O9A3T:00502:02153,AAAGATGAGCTCTCGG-1,True,137,A1072B310,sample,BC-328,"A0301, A2402","B0702, B0801","C0701, C0702",PBMC,"[A0301, A2402, B0702, B0801, C0701, C0702]"
8990,1,106,0,137,A1072B313_sample,O9A3T:00631:04877 BX:Z:AAACCTGAGCACCGTC-1,O9A3T:00631:04877,AAACCTGAGCACCGTC-1,True,137,A1072B313,sample,BC-329,"A0201, A2501","B0702, B3501","C0401, C0702",PBMC,"[A0201, A2501, B0702, B3501, C0401, C0702]"
8991,1,106,0,137,A1072B313_sample,O9A3T:01687:04706 BX:Z:AAACCTGAGCACCGTC-1,O9A3T:01687:04706,AAACCTGAGCACCGTC-1,True,137,A1072B313,sample,BC-329,"A0201, A2501","B0702, B3501","C0401, C0702",PBMC,"[A0201, A2501, B0702, B3501, C0401, C0702]"
8992,1,106,0,137,A1072B313_sample,O9A3T:01684:02332 BX:Z:AAACCTGAGCACCGTC-1,O9A3T:01684:02332,AAACCTGAGCACCGTC-1,True,137,A1072B313,sample,BC-329,"A0201, A2501","B0702, B3501","C0401, C0702",PBMC,"[A0201, A2501, B0702, B3501, C0401, C0702]"
8993,1,106,0,137,A1072B313_sample,O9A3T:01675:02294 BX:Z:AAACCTGAGCACCGTC-1,O9A3T:01675:02294,AAACCTGAGCACCGTC-1,True,137,A1072B313,sample,BC-329,"A0201, A2501","B0702, B3501","C0401, C0702",PBMC,"[A0201, A2501, B0702, B3501, C0401, C0702]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15163,8,37,28,88,A1072B314_sample,O9A3T:01164:05923 BX:Z:AAACGGGTCCCAGGTG-1,O9A3T:01164:05923,AAACGGGTCCCAGGTG-1,False,60,A1072B314,sample,BC-337,"A0201, A0301","B1402, B4001","C0304, C0802",PBMC,"[A0201, A0301, B1402, B4001, C0304, C0802]"
15164,1,77,28,137,A1072B314_sample,O9A3T:00643:01273 BX:Z:AAACGGGTCCCAGGTG-1,O9A3T:00643:01273,AAACGGGTCCCAGGTG-1,True,109,A1072B314,sample,BC-337,"A0201, A0301","B1402, B4001","C0304, C0802",PBMC,"[A0201, A0301, B1402, B4001, C0304, C0802]"
15165,1,77,28,137,A1072B314_sample,O9A3T:00298:02608 BX:Z:AAACGGGTCCCAGGTG-1,O9A3T:00298:02608,AAACGGGTCCCAGGTG-1,True,109,A1072B314,sample,BC-337,"A0201, A0301","B1402, B4001","C0304, C0802",PBMC,"[A0201, A0301, B1402, B4001, C0304, C0802]"
15166,8,30,28,82,A1072B314_sample,O9A3T:00174:05779 BX:Z:AAACGGGTCCCAGGTG-1,O9A3T:00174:05779,AAACGGGTCCCAGGTG-1,False,54,A1072B314,sample,BC-337,"A0201, A0301","B1402, B4001","C0304, C0802",PBMC,"[A0201, A0301, B1402, B4001, C0304, C0802]"


In [16]:
df.HLA_match.dropna().sum() / len(df.HLA_match.dropna())

0.29276315789473684

In [22]:
df[(df.single_barcode_cd8 == True) & (df.single_barcode_mhc == False)].peptide_HLA_lst[1]

"['GPFPGNLVD B0702' 'WAGGLGLNL B0702' 'VTEHDTLLY A0101']"

In [25]:
df.groupby('gem').HLA_cd8.apply(list).to_dict()

{'AAACCTGAGAGGTAGA-1': [nan],
 'AAACCTGAGCACCGTC-1': ["['A0201', 'A2501', 'B0702', 'B3501', 'C0401', 'C0702']"],
 'AAACCTGAGGAGCGTT-1': [nan],
 'AAACCTGAGTCGAGTG-1': [nan],
 'AAACCTGCATACTACG-1': [nan],
 'AAACCTGGTATAAACG-1': [nan],
 'AAACCTGGTTTGACTG-1': [nan],
 'AAACCTGTCGAATCCA-1': [nan],
 'AAACGGGAGAGGACGG-1': [nan],
 'AAACGGGAGGACATTA-1': [nan],
 'AAACGGGCACCAGATT-1': [nan],
 'AAACGGGCATCGGACC-1': [nan],
 'AAACGGGGTGATGTCT-1': [nan],
 'AAACGGGGTTAGGGTG-1': [nan],
 'AAACGGGTCCCAGGTG-1': ["['A0201', 'A0301', 'B1402', 'B4001', 'C0304', 'C0802']"],
 'AAAGATGAGAGGTTGC-1': [nan],
 'AAAGATGGTACTCTCC-1': [nan],
 'AAAGCAAAGCTCTCGG-1': [nan],
 'AAAGCAACAAGCTGAG-1': [nan],
 'AAAGCAAGTACATGTC-1': [nan],
 'AAAGCAAGTGAGTATA-1': [nan],
 'AAAGCAAGTTTACTCT-1': [nan],
 'AAAGTAGAGCTCAACT-1': [nan],
 'AAAGTAGCAGTACACT-1': [nan],
 'AAAGTAGTCGTAGGAG-1': [nan],
 'AAATGCCCATGACGGA-1': [nan],
 'AAATGCCGTACAAGTA-1': [nan],
 'AAATGCCGTATGGTTC-1': [nan],
 'AAATGCCTCCAAAGTC-1': [nan],
 'AAATGCCTCTCTAAGG-1': [

In [37]:
df.peptide_likely.dropna()

Series([], Name: peptide_likely, dtype: float64)

In [6]:
df.columns

Index(['gem', 'clonotype', 'num_clonotype', 'ct', 'genes_TRA', 'genes_TRB',
       'genes_lst_TRA', 'genes_lst_TRB', 'length_TRA', 'cdr3_TRA',
       'umi_count_TRA', 'umi_count_lst_TRA', 'delta_umi_TRA', 'cdr3_lst_TRA',
       'chain_count_TRA', 'single_TRA', 'length_TRB', 'cdr3_TRB',
       'umi_count_TRB', 'umi_count_lst_TRB', 'delta_umi_TRB', 'cdr3_lst_TRB',
       'chain_count_TRB', 'single_TRB', 'single_chain_only', 'umi_count_tcr',
       'cdr3_comb', 'tcr_category', 'no_filtration',
       'exclude_single-chain_TCRs', 'exclude_ambiguous_and_single-chain_TCRs',
       'template_id_mhc', 'template_lst_mhc', 'single_barcode_mhc',
       'umi_count_mhc', 'umi_count_lst_mhc', 'read_count_mhc',
       'read_count_lst_mhc', 'multiplets_mhc', 'delta_umi_mhc',
       'template_id_cd8', 'template_lst_cd8', 'single_barcode_cd8',
       'umi_count_cd8', 'umi_count_lst_cd8', 'read_count_cd8',
       'read_count_lst_cd8', 'detected_response', 'peptide_assayed',
       'sample_id', 'sample_id

In [11]:
df.HLA_mhc.unique()

array(['B0801', 'A0101', 'B0702', 'A0201', 'A2401'], dtype=object)

In [10]:
cf = pd.DataFrame(index=df.HLA_mhc.unique(), columns=df.HLA_mhc.unique())

In [13]:
cf.fillna(0, inplace=True)

In [15]:
cf

,B0801,A0101,B0702,A0201,A2401
B0801,0,0,0,0,0
A0101,0,0,0,0,0
B0702,0,0,0,0,0
A0201,0,0,0,0,0
A2401,0,0,0,0,0


In [ ]:
df.apply(lambda row: cf.loc[row.HLA_mhc, row.HLA_mhc] += 1 if row.HLA_mch.isin(row.HLA_cd8), axis=1)

In [88]:
df[(df.peptide_HLA == 'EERQAYLTNQY A0101') & (df.ct == 4)]

,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,cdr3_TRA,...,peptide_lst,peptide_likely,peptide_HLA,peptide_HLA_lst,epitope,epitope_lst,gems_per_specificity,gems_per_clonotype,binding_concordance,test
2270,GAGTCCGAGGCGACAT-1,clonotype2,2.0,4.0,NaN,TRBV7-3;;TRBJ2-1;TRBC2,NaN,['TRBV7-3;;TRBJ2-1;TRBC2'],NaN,NaN,...,['VTEHDTLLY' 'EERQAYLTNQY'],NaN,EERQAYLTNQY A0101,['VTEHDTLLY A0101' 'EERQAYLTNQY A0101'],45_1_3,['V15_A1 CMV pp50 VTE' '45_1_3'],19.0,574.0,0.033101,"[a, b, c]"


In [ ]:
flt_df.drop(flt_df[~flt_df.duplicated(subset=[clonotype_fmt, 'peptide_HLA'], keep=False)].index, inplace=True)

In [101]:
df[~df.duplicated(subset=['ct','peptide_HLA'], keep=False)].sort_values(by=['ct','peptide_HLA','gem']).head(60)

,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,cdr3_TRA,...,peptide_lst,peptide_likely,peptide_HLA,peptide_HLA_lst,epitope,epitope_lst,gems_per_specificity,gems_per_clonotype,binding_concordance,test
1339,GCGCAGTGTAAACCTC-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],510.0,CAASQNEKLTF,...,['VPAQRLQEA' 'SLIYPLVKM' 'ILKEPVHGV' 'VTEHDTLL...,NaN,DEYVHNTTA B0801,['VPAQRLQEA B0801' 'SLIYPLVKM B0801' 'ILKEPVHG...,47_1_79,['47_1_70' '47_1_33' 'V41_A2 HIV Pol (C20) (IL...,1.0,496.0,0.002016,"[a, b, c]"
830,CGATGTAGTCAACTGT-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],540.0,CAASQNEKLTF,...,['FLDTNQVV'],NaN,FLDTNQVV A0201,['FLDTNQVV A0201'],06_1_3,['06_1_3'],1.0,496.0,0.002016,"[a, b, c]"
927,CGTCAGGTCACCTCGT-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],541.0,CAASQNEKLTF,...,['FLVNTMCLAL'],NaN,FLVNTMCLAL A0201,['FLVNTMCLAL A0201'],06_1_8,['06_1_8'],1.0,496.0,0.002016,"[a, b, c]"
1375,GCTGGGTGTACGACCC-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],510.0,CAASQNEKLTF,...,['GMAEGATTA'],NaN,GMAEGATTA A0101,['GMAEGATTA A0101'],45_1_37,['45_1_37'],1.0,496.0,0.002016,"[a, b, c]"
1329,GCGAGAAAGACTGTAA-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],512.0,CAASQNEKLTF,...,['AYRAALGASL' 'RAKFKQLL' 'VTEHDTLLY' 'GPKPAKPC...,NaN,GPFPGNLVD B0702,['AYRAALGASL B0702' 'RAKFKQLL B0801' 'VTEHDTLL...,47_1_38,['47_1_74' 'V17_B8 EBV BZLF1 (C9)' 'V15_A1 CM...,1.0,496.0,0.002016,"[a, b, c]"
1054,CTCTACGAGGCTACGA-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],478.0,CAASQNEKLTF,...,['GPFPGNLVDV'],NaN,GPFPGNLVDV B0702,['GPFPGNLVDV B0702'],47_1_39,['47_1_39'],1.0,496.0,0.002016,"[a, b, c]"
747,CCTATTAGTCCTAGCG-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],514.0,CAASQNEKLTF,...,['LHPSTPLKPL' 'GPFPGNLVD' 'VTEHDTLLY' 'WPRAPRG...,NaN,GPKPAKPCSGA B0702,['LHPSTPLKPL B0702' 'GPFPGNLVD B0702' 'VTEHDTL...,47_1_41,['47_1_53' '47_1_38' 'V15_A1 CMV pp50 VTE' ...,1.0,496.0,0.002016,"[a, b, c]"
735,CCTAAAGCAATTGCTG-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],475.0,CAASQNEKLTF,...,['ICSSVDIHF'],NaN,ICSSVDIHF A0101,['ICSSVDIHF A0101'],45_1_38,['45_1_38'],1.0,496.0,0.002016,"[a, b, c]"
1086,CTGCGGAAGTGTACGG-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],514.0,CAASQNEKLTF,...,['IRHKRQSWV'],NaN,IRHKRQSWV B0801,['IRHKRQSWV B0801'],47_1_94,['47_1_94'],1.0,496.0,0.002016,"[a, b, c]"
897,CGGCTAGGTTCCACTC-1,clonotype1,1.0,1.0,TRAV21;TRAJ48;TRAC,TRBV27;;TRBJ1-6;TRBC1,['TRAV21;TRAJ48;TRAC'],['TRBV27;;TRBJ1-6;TRBC1'],510.0,CAASQNEKLTF,...,['LFSAGAFILYN'],NaN,LFSAGAFILYN A0101,['LFSAGAFILYN A0101'],45_1_8,['45_1_8'],1.0,496.0,0.002016,"[a, b, c]"
